In [ ]:
#load path
import os
import sys
import astropy
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table

from astropy.wcs import WCS
from photutils.aperture import CircularAperture, aperture_photometry, CircularAnnulus
from astropy.visualization import (ImageNormalize, MinMaxInterval,PercentileInterval,SqrtStretch)
import glob


In [ ]:
path='./data/'
rawpath=r'/Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/'
calibpath='./data/Calibs/'
reduxpath='./data/Redux/'
combinepath='./data/Redux/Combined/'
gain=0.25 #e-/ADU at G = 125

In [ ]:
def makeflat(flatlist,masterbias,filter):
    flatshape=fits.open(flatlist[0])[0].data
    masterflat=np.zeros((len(flatlist),flatshape.shape[0],flatshape.shape[1]))
    for ii,img in enumerate(flatlist):
        hdu=fits.open(img)
        masterflat[ii,:,:]=hdu[0].data-masterbias
    masterflat=np.median(masterflat,axis=0)
    normflat=masterflat/np.median(masterflat)
    return normflat

In [ ]:
nightbias = '2025-09-25/BIAS/'
biaslist=glob.glob(rawpath+nightbias+'*BIAS*.fits')
print('Number of bias frames found: {}'.format(len(biaslist)))


In [ ]:
#Now process bias frames
biasshape=fits.open(biaslist[0])[0].data
masterbias=np.zeros((len(biaslist),biasshape.shape[0],biasshape.shape[1]))
print(masterbias.shape)

for ii,img in enumerate(biaslist):
    hdu=fits.open(img)
    masterbias[ii,:,:]=hdu[0].data

masterbias=np.median(masterbias,axis=0)
#display master bias
plt.imshow(masterbias,cmap='gray',norm=ImageNormalize(masterbias,interval=MinMaxInterval(),stretch=SqrtStretch()))
#save as fits
fits.writeto(calibpath+'/MasterBias.fits',masterbias,overwrite=True)



In [ ]:
#Now process darks as a function of exptime
nightdark = '2025-09-25/DARK/'
darklist=glob.glob(rawpath+nightdark+'*DARK*.fits')
darkshape=fits.open(darklist[0])[0].data
masterdark=np.zeros((len(darklist),darkshape.shape[0],darkshape.shape[1]))
darkexptimes=np.zeros(len(darklist))
for ii,img in enumerate(darklist):
    hdu=fits.open(img)
    darkexptimes[ii]=hdu[0].header['EXPTIME']
    masterdark[ii,:,:]=hdu[0].data-masterbias
#find unique exptimes
uniqexptimes=np.unique(darkexptimes)
print(uniqexptimes)
meddarkvals = np.zeros((len(uniqexptimes)))
for exptime in uniqexptimes:
    darks=masterdark[darkexptimes==exptime,:,:]
    meddark=np.median(darks,axis=0)
    #plt.imshow(meddark,cmap='gray',norm=ImageNormalize(meddark,interval=MinMaxInterval(),stretch=SqrtStretch()))
    fits.writeto(calibpath+'/MasterDark_'+str(int(exptime))+'s.fits',meddark,overwrite=True)
    meddarkvals[uniqexptimes==exptime]=np.median(meddark)

print(meddarkvals)


In [ ]:
#fit a line to the median dark values vs exptime
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(uniqexptimes,meddarkvals)
print('Dark current = {:6.4f} e-/s '.format(slope*gain))
plt.plot(uniqexptimes,meddarkvals,'o')
plt.plot(uniqexptimes,slope*uniqexptimes+intercept)
plt.xlabel('Exposure time (s)')
plt.ylabel('Median dark (e-)')
plt.title('Dark current = {:6.4f} e-/s'.format(slope*gain))

In [ ]:
#Now process flats
nightflat = '2025-10-01/FLAT/'
for filter in ['R','G','I','SII','OIII','Ha','Hb']:
    flatlist=glob.glob(rawpath+nightflat+'/*_FLAT*'+filter+'*.fits')
    normflat=makeflat(flatlist,masterbias,filter)
    plt.imshow(normflat,cmap='gray',norm=ImageNormalize(normflat,interval=MinMaxInterval(),stretch=SqrtStretch()))
    fits.writeto(calibpath+'/MasterFlat_'+filter+'.fits',normflat,overwrite=True)